In [2]:
from pymilvus_orm import connections, FieldSchema, CollectionSchema, DataType, Collection

In [3]:
connections.connect(host='localhost', port='19530')

In [4]:
dim = 128
default_fields = [
    FieldSchema(name="count", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="random_value", dtype=DataType.DOUBLE),
    FieldSchema(name="float_vector", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
default_schema = CollectionSchema(fields=default_fields, description="test collection")

print(f"\nCreate collection...")
collection = Collection(name="hello_milvus", schema=default_schema)


Create collection...


In [5]:
collection

{"name": "hello_milvus", "schema": {"auto_id": false, "description": "test collection", "fields": [{"name": "count", "description": "", "type": 5, "is_primary": true, "auto_id": false}, {"name": "random_value", "description": "", "type": 11}, {"name": "float_vector", "description": "", "type": 101, "params": {"dim": 128}}]}, "partitions": [{"name": "_default", "collection_name": "hello_milvus", "description": ""}], "description": "test collection"}

In [6]:
type(collection)

pymilvus_orm.collection.Collection

In [7]:
import random
nb = 3000
vectors = [[random.random() for _ in range(dim)] for _ in range(nb)]
collection.insert(
    [
        [i for i in range(nb)],
        [float(random.randrange(-20,-10)) for _ in range(nb)],
        vectors
    ]
)

(insert count: 3000, delete count: 0, upsert count: 0, timestamp: 427286366892326913)

In [8]:
default_index = {"index_type": "IVF_FLAT", "params": {"nlist": 128}, "metric_type": "L2"}
collection.create_index(field_name="float_vector", index_params=default_index)
collection.load()

In [9]:
topK = 5
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
# define output_fields of search result
res = collection.search(
    vectors[-2:], "float_vector", search_params, topK,
    "count > 100", output_fields=["count", "random_value"]
)

In [10]:
res

In [11]:
vars(res)

{'_qs': <pymilvus.client.abstract.ChunkedQueryResult at 0x7f2fcf27ffd0>}

In [12]:
list(res)

In [14]:
hit = res[0]

In [15]:
vars(hit)

{'_LoopBase__index': 0,
 '_raw': fields_data {
   type: Int64
   field_name: "count"
   scalars {
     long_data {
       data: 2998
       data: 1366
       data: 187
       data: 2667
       data: 2993
     }
   }
 }
 fields_data {
   type: Double
   field_name: "random_value"
   scalars {
     double_data {
       data: -12.0
       data: -15.0
       data: -17.0
       data: -20.0
       data: -15.0
     }
   }
 }
 scores: 0.0
 scores: 14.647258758544922
 scores: 14.935089111328125
 scores: 15.197955131530762
 scores: 15.30621337890625
 ids {
   int_id {
     data: 2998
     data: 1366
     data: 187
     data: 2667
     data: 2993
   }
 },
 '_auto_id': False,
 '_distances': [0.0, 14.647258758544922, 14.935089111328125, 15.197955131530762, 15.30621337890625],
 '_entities': [<pymilvus.client.abstract.Hit at 0x7f2fcf2968b0>,
  <pymilvus.client.abstract.Hit at 0x7f2fcf296130>]}